In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:55<00:00, 21.7MB/s]
100% 1.06G/1.06G [00:55<00:00, 20.5MB/s]


In [ ]:
import zipfile

zipref = zipfile.ZipFile('/content/dogs-vs-cats.zip')
zipref.extractall('/content')
zipref.close()

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout,Input

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256, 256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256, 256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [ ]:
def process(image, label):
    image = tensorflow.cast(image/255., tensorflow.float32)
    image = tensorflow.image.resize(image, (256, 256))
    return image, label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
model = Sequential()

model.add(Input(shape=(256,256,3)))

model.add(Conv2D(32, kernel_size=(3, 3),padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64, kernel_size=(3, 3),padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128, kernel_size=(3, 3),padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,847,297 (56.64 MB)

 Trainable params: 14,847,297 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data = validation_ds)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 76s 107ms/step - accuracy: 0.5914 - loss: 0.6945 - val_accuracy: 0.6856 - val_loss: 0.6105
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 67s 107ms/step - accuracy: 0.7314 - loss: 0.5401 - val_accuracy: 0.7608 - val_loss: 0.5144
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 74s 95ms/step - accuracy: 0.8066 - loss: 0.4201 - val_accuracy: 0.7790 - val_loss: 0.5353
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 78s 89ms/step - accuracy: 0.8836 - loss: 0.2779 - val_accuracy: 0.7734 - val_loss: 0.7348
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 88s 98ms/step - accuracy: 0.9418 - loss: 0.1456 - val_accuracy: 0.7802 - val_loss: 0.9045
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 58s 93ms/step - accuracy: 0.9662 - loss: 0.0927 - val_accuracy: 0.7572 - val_loss: 0.9989
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 64s 102ms/step - accuracy: 0.9777 - loss: 0.0661 - val_accuracy: 0.7730 - val_loss: 0.9721
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 73s 116ms/step - accuracy: 0.9811 - loss: 0.057

# Data Augementation

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
batch_size = 32

train_image_generator = ImageDataGenerator(
    rotation_range = 0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    rescale = 1./255,
    horizontal_flip = True,
)

validation_image_generator = ImageDataGenerator(
    rescale = 1./255
)

train_generator = train_image_generator.flow_from_directory(
    directory = '/content/train',
    target_size = (256, 256),
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = True,
    seed = 42
)
test_generator = validation_image_generator.flow_from_directory(
    directory = '/content/test',
    target_size = (256, 256),
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = True,
    seed = 42
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# New generated image use in training. Orginal keep as it is
model.fit(train_generator,epochs = 10,validation_data = test_generator)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 327s 511ms/step - accuracy: 0.7104 - loss: 0.6337 - val_accuracy: 0.8070 - val_loss: 0.4355
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 328s 520ms/step - accuracy: 0.7781 - loss: 0.4744 - val_accuracy: 0.8150 - val_loss: 0.4061
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 329s 521ms/step - accuracy: 0.7905 - loss: 0.4534 - val_accuracy: 0.8214 - val_loss: 0.3930
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 390s 531ms/step - accuracy: 0.8024 - loss: 0.4353 - val_accuracy: 0.8456 - val_loss: 0.3512
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 327s 518ms/step - accuracy: 0.8160 - loss: 0.4057 - val_accuracy: 0.8464 - val_loss: 0.3522
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 390s 531ms/step - accuracy: 0.8277 - loss: 0.3918 - val_accuracy: 0.8438 - val_loss: 0.3523
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 368s 508ms/step - accuracy: 0.8413 - loss: 0.3616 - val_accuracy: 0.8556 - val_loss: 0.3364
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 321s 509ms/step - accuracy: 0.8415 - loss: 0.35

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to preprocess the image
def preprocess_image(img_path, target_size=(256, 256)):
    img = image.load_img(img_path, target_size=target_size)  # Load the image and resize it
    img_array = image.img_to_array(img)  # Convert the image to a numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.  # Rescale the image (same as in your training)
    return img_array

# Path to the new image
img_path = '/content/images.jpg'

# Preprocess the image
new_image = preprocess_image(img_path)

# Make prediction
prediction = model.predict(new_image)

# Interpret the prediction
if prediction[0] > 0.5:
    print("Prediction: This is the Image Of Cat")
else:
    print("Prediction: This is the Image of Dog")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Prediction: This is the Image Of Dog


In [ ]:
# Save the model using the Keras format
model.save('my_model.keras')


In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('my_model.keras')